In [1]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/cambrian/'
data_folder = parent_folder + 'data/'
from hydra import compose, initialize
with initialize(config_path="../../conf"):
    # Compose the configuration
    cfg = compose(config_name="ClustersResetService.yaml")

import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_6057/1083430745.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="../../conf"):


In [9]:
import pandas as pd

In [2]:
import sys

sys.path.append('/Users/username/Desktop/ML/Recommendations/cambrian/')

import hydra
import os
from hydra import compose, initialize

In [4]:
from pymongo import MongoClient

In [3]:
class MongoDatabaseConnection:
    _instance = None
    _client = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(MongoDatabaseConnection, cls).__new__(cls)
            cls._instance._client = MongoClient(
                host='localhost',
                port=27017
            )
        return cls._instance

    @classmethod
    def get_database(cls, db_name=None):
        db_name = 'insight_db'
        return cls._instance._client[db_name]

In [5]:
db = MongoDatabaseConnection().get_database()

In [6]:
articles_collection = db['articles']

In [7]:
articles = articles_collection.find(
    {"is_premium_article": False},
    { "published_time": 1, "source_id": 1, "_id": 1, 'url': 1}
)

In [10]:
df = pd.DataFrame(list(articles))

In [12]:
df = df.rename(columns = {'_id': 'article_id'})

In [56]:
df['article_id'] = df['article_id'].astype('str')
df['source_id'] = df['source_id'].astype('str')

In [58]:
def extract_cat_sub_cat_from_url(row):
    # ET
    if row['source_id'] == '6512cdcad01a9c8e86263e05':
        cat = row['url'].split('economictimes.indiatimes.com/')[1].split('/')[0]
        sub_cat = row['url'].split('economictimes.indiatimes.com/')[1].split('/')[1]
        return cat, sub_cat
    # livemint
    elif row['source_id'] == '650046bd005149c49201269f':
        cat = row['url'].split('livemint.com/')[1].split('/')[0]
        sub_cat = row['url'].split('livemint.com/')[1].split('/')[1]
        return cat, sub_cat

In [57]:
extract_cat_sub_cat_from_url(df.iloc[0])

<class 'str'>
yes
news defence


('news', 'defence')

In [33]:
s = df.head(1).apply(extract_cat_sub_cat_from_url, axis = 1)

https://economictimes.indiatimes.com/news/defence/israel-declares-war-bombards-gaza-and-battles-to-dislodge-hamas-fighters-after-surprise-attack/articleshow/104269818.cms


In [39]:
s = df.head(1)['url'].values[0]

In [45]:
s.split('economictimes.indiatimes.com/')[1].split('/')[1]

'defence'

In [59]:
df['category'], df['sub_category'] = zip(*df.apply(extract_cat_sub_cat_from_url, axis = 1))
# articles['sub_category'] = articles.apply(lambda x: x.split('https://economictimes.indiatimes.com/')[1].split('/')[1])

In [61]:
df['combined_cat'] = df['category'] + '-' + df['sub_category']

In [64]:
df.groupby('category')['category'].count()

category
industry      2504
markets        938
news         19248
nri              6
small-biz      110
wealth          73
Name: category, dtype: int64

In [82]:
whitelisted_categories = ['industry', 'markets', 'nri', 'small-biz', 'wealth']
whitelisted_sub_cats = ['news-company', 'news-economy', 'news-industry']

In [89]:
df['is_relevant'] = df.apply(lambda x: x['category'] in whitelisted_categories or x['sub_category'] in whitelisted_sub_cats, axis = 1)

In [91]:
df.is_relevant.sum()

3631

In [65]:
df[df.category == 'news'].groupby('sub_category')['category'].count()

sub_category
company                                                                                                                               84
defence                                                                                                                             1142
economy                                                                                                                              677
elections                                                                                                                            484
environment                                                                                                                           75
et-india-rising-india-on-track-to-be-a-5-trillion-economy-by-2025-says-sbi-chairman-dinesh-khara                                       1
get-ready-to-experience-the-best-of-oneplus-with-the-nord-ce-3-lite-5g-and-nord-buds-2-sale-exciting-offers-that-you-cannot-miss       1
gold-rises-rs-195-silver-up-

In [63]:
df.groupby('combined_cat')['combined_cat'].count()

combined_cat
industry-auto                   219
industry-banking                295
industry-cons-products          214
industry-csr                      3
industry-energy                 174
                               ... 
wealth-personal-finance-news     29
wealth-plan                       2
wealth-real-estate                3
wealth-save                       2
wealth-tax                        1
Name: combined_cat, Length: 62, dtype: int64

In [62]:
df

,article_id,url,published_time,source_id,category,sub_category,combined_cat
0,65236ceccd871c6ac1cde043,https://economictimes.indiatimes.com/news/defe...,2023-10-09T07:06:00+05:30,6512cdcad01a9c8e86263e05,news,defence,news-defence
1,65236cf1cd871c6ac1cde059,https://economictimes.indiatimes.com/news/spor...,2023-10-09T06:59:00+05:30,6512cdcad01a9c8e86263e05,news,sports,news-sports
2,65236ce5cd871c6ac1cde032,https://economictimes.indiatimes.com/news/inte...,2023-10-09T06:52:00+05:30,6512cdcad01a9c8e86263e05,news,international,news-international
3,65236cefcd871c6ac1cde04c,https://economictimes.indiatimes.com/markets/i...,2023-10-09T06:15:00+05:30,6512cdcad01a9c8e86263e05,markets,ipos,markets-ipos
4,65236cdfcd871c6ac1cde02b,https://economictimes.indiatimes.com/news/inte...,2023-10-09T02:17:00+05:30,6512cdcad01a9c8e86263e05,news,international,news-international
...,...,...,...,...,...,...,...
22874,65221b66cd871c6ac1cde01c,https://economictimes.indiatimes.com/news/inte...,2023-10-08T00:30:00+05:30,6512cdcad01a9c8e86263e05,news,international,news-international
22875,65221b5dcd871c6ac1cde011,https://economictimes.indiatimes.com/news/inte...,2023-10-08T00:18:00+05:30,6512cdcad01a9c8e86263e05,news,international,news-international
22876,65221b68cd871c6ac1cde021,https://economictimes.indiatimes.com/news/elec...,2023-10-08T00:46:00+05:30,6512cdcad01a9c8e86263e05,news,elections,news-elections
22877,65221b67cd871c6ac1cde01d,https://economictimes.indiatimes.com/news/inte...,2023-10-08T00:23:00+05:30,6512cdcad01a9c8e86263e05,news,international,news-international


### Sandbox

In [68]:
import numpy as np

In [69]:
def calculate_exploit_level(mins_since_last_api):
    if mins_since_last_api <= 1440:
        return np.ceil(2 * mins_since_last_api / 480) / 2
    else:
        days_since_last_api = np.ceil(2 * mins_since_last_api / 1440) / 2
        return max(3, np.clip(days_since_last_api, 3, 10))

In [79]:
def compute_probability_powers():
    probability_powers = {}
    for i in range(1001):  # 0.00, 0.01, ..., 1.00
        prob = i / 1000.0
        probability_powers[prob] = {}
        for power in [x * 0.5 for x in range(1, 41)]:
            probability_powers[prob][power] = np.round(prob ** power, 3)
    return probability_powers

In [80]:
powers = compute_probability_powers()

In [81]:
powers

{0.0: {0.5: 0.0,
  1.0: 0.0,
  1.5: 0.0,
  2.0: 0.0,
  2.5: 0.0,
  3.0: 0.0,
  3.5: 0.0,
  4.0: 0.0,
  4.5: 0.0,
  5.0: 0.0,
  5.5: 0.0,
  6.0: 0.0,
  6.5: 0.0,
  7.0: 0.0,
  7.5: 0.0,
  8.0: 0.0,
  8.5: 0.0,
  9.0: 0.0,
  9.5: 0.0,
  10.0: 0.0,
  10.5: 0.0,
  11.0: 0.0,
  11.5: 0.0,
  12.0: 0.0,
  12.5: 0.0,
  13.0: 0.0,
  13.5: 0.0,
  14.0: 0.0,
  14.5: 0.0,
  15.0: 0.0,
  15.5: 0.0,
  16.0: 0.0,
  16.5: 0.0,
  17.0: 0.0,
  17.5: 0.0,
  18.0: 0.0,
  18.5: 0.0,
  19.0: 0.0,
  19.5: 0.0,
  20.0: 0.0},
 0.001: {0.5: 0.032,
  1.0: 0.001,
  1.5: 0.0,
  2.0: 0.0,
  2.5: 0.0,
  3.0: 0.0,
  3.5: 0.0,
  4.0: 0.0,
  4.5: 0.0,
  5.0: 0.0,
  5.5: 0.0,
  6.0: 0.0,
  6.5: 0.0,
  7.0: 0.0,
  7.5: 0.0,
  8.0: 0.0,
  8.5: 0.0,
  9.0: 0.0,
  9.5: 0.0,
  10.0: 0.0,
  10.5: 0.0,
  11.0: 0.0,
  11.5: 0.0,
  12.0: 0.0,
  12.5: 0.0,
  13.0: 0.0,
  13.5: 0.0,
  14.0: 0.0,
  14.5: 0.0,
  15.0: 0.0,
  15.5: 0.0,
  16.0: 0.0,
  16.5: 0.0,
  17.0: 0.0,
  17.5: 0.0,
  18.0: 0.0,
  18.5: 0.0,
  19.0: 0.0,
  19.5:

In [78]:
calculate_exploit_level(500)

1.5